In [ ]:
# https://www.properati.com.ar/data
! wget -O ar_properties.csv.gz -q https://storage.googleapis.com/properati-data-public/ar_properties.csv.gz
! gzip -d -f /content/ar_properties.csv.gz

In [ ]:
import pandas as pd

import sklearn as sk
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics

# Lectura de datos

In [ ]:
df = pd.read_csv('/content/ar_properties.csv', index_col="id")  

In [ ]:
# seleccion de datos de demostración. cambiar
df = df.loc[(df.l2 == "Córdoba") & (df.operation_type == 'Venta') & (df.property_type == 'Casa')]
df.shape

# Análisis del conjunto de datos

# Tratamiento de los datos

# Modelización

In [ ]:
df = df.select_dtypes(include=['float64', 'int64'])
df.fillna(0, inplace=True, downcast= "infer")

In [ ]:
X = df[df.columns.drop('price')]
y = df['price']

for n_estimators in [100, 500, 1000]:     
    for max_depth in [3, 7, 11]:

        ## Tienen que usar RandomForestRegressor si o si o si. Pueden cambiar los parámetros
        reg = sk.ensemble.RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42, n_jobs=-1)

        ## Validación cruzada en 5 partes (lo van a ver en AA), -RMSE. No tocar
        scores = sk.model_selection.cross_val_score(reg, X, y, cv=10, scoring='neg_root_mean_squared_error')

        ## Imprimimos scores. Cuando más bajo mejor
        print(f"n_estimators={n_estimators}, max_depth={max_depth} --> {-scores.mean():.3f} +/- {scores.std():.3f}")


In [ ]:
df_pred = pd.read_csv('/content/a_predecir.csv', index_col="id")
df_pred.fillna(0, inplace=True)

In [ ]:
X_pred = df_pred[X.columns]

In [ ]:
# los mejores hiperparámetros encontrados antes
n_estimators = 100
max_depth = 3

# entrenamiento
reg = sk.ensemble.RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42, n_jobs=-1)
reg.fit(X, y)

# predicción
df_pred["price"] = reg.predict(X_pred)

In [ ]:
# grabo la solución
df_pred[["price"]].to_csv("solucion.csv")